In [1]:
import tensorflow as tf
import bert4keras
import keras
import pandas as pd
import os
print(tf.__version__)
print(keras.__version__)
print(bert4keras.__version__)
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open, ViterbiDecoder, to_array
from bert4keras.layers import ConditionalRandomField
from keras.layers import Dense
from keras.models import Model
from tqdm import tqdm
from utils import utils
from imp import reload
reload(utils)

/Users/mac/opt/anaconda3/envs/py_tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/mac/opt/anaconda3/envs/py_tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/mac/opt/anaconda3/envs/py_tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/mac/opt/anaconda3/envs/py_tf/lib/pyt

1.14.0
2.3.1
0.9.2


<module 'utils.utils' from '/Users/mac/Desktop/MLLearn/比赛/CCF隐私/course/13th_baseline介绍/DBC_code/src/utils/utils.py'>

In [ ]:
# 预训练模型的超参数
maxlen = 300
epochs = 15
batch_size = 8
bert_layers = 12
learing_rate = 1e-5 
crf_lr_multiplier = 1000 
rnn_lr_multiplier = 1000
# 数据处理参数
symbol = ['？','⋯','…','﹗']
max_sent_length = 250
max_input_length = 300     
# bert配置
config_path = '../model/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../model/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../model/chinese_L-12_H-768_A-12/vocab.txt'
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 类别映射
labels = [
    'position',
    'name',
    'organization',
    'company',
    'address',
    'movie',
    'game',
    'government',
    'scene',
    'book',
    'mobile',
    'email',
    'QQ',
    'vx',
]

# 0 表示 'O'
# 其他数字表示对应的 B 和 I
id2label = dict(enumerate(labels))
label2id = {j: i for i, j in id2label.items()}
num_labels = len(labels) * 2 + 1

In [ ]:
model = build_transformer_model(
    config_path,
    checkpoint_path,
)

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model.get_layer(output_layer).output
output = Dense(num_labels)(output)

In [ ]:
CRF = ConditionalRandomField(lr_multiplier=crf_lr_multiplier)
output = CRF(output)
model = Model(model.input, output)
model.summary()

In [ ]:
model_path = '../model/best_model_epoch_10.weights'
model.load_weights(model_path)

In [ ]:
class NamedEntityRecognizer(ViterbiDecoder):
    """
    命名实体识别器
    """
    def recognize(self, text):
        tokens = tokenizer.tokenize(text)
        mapping = tokenizer.rematch(text, tokens)
        token_ids = tokenizer.tokens_to_ids(tokens)
        segment_ids = [0] * len(token_ids)
        nodes = model.predict([[token_ids], [segment_ids]])[0]
        labels = self.decode(nodes)
        entities, starting = [], False
        
        for i, label in enumerate(labels):
            if label > 0:
                if label % 2 == 1:
                    starting = True
                    entities.append([[i], id2label[(label - 1) // 2]])
                elif starting:
                    entities[-1][0].append(i)
                else:
                    starting = False
            else:
                starting = False

        return [(text[mapping[w[0]][0]:mapping[w[-1]][-1] + 1], l)
                for w, l in entities]

    
    
NER = NamedEntityRecognizer(trans=K.eval(CRF.trans), starts=[0], ends=[0])

In [ ]:
def test_predict(data, NER_):
    test_ner =[]
    for text in tqdm(data):
        cut_text_list, cut_index_list = utils.agg_sent([text],symbol, max_sent_length, max_input_length)
        posit = 0
        item_ner = []
        index =1
        for str_ in cut_text_list:
            ner_res  = NER_.recognize(str_)
            for tn in ner_res:
                ans = {}
                ans["label_type"] = tn[1]
                ans['index'] = str(index)
                ans["start_pos"] = text.find(tn[0],posit)
                ans["end_pos"] = ans["start_pos"] + len(tn[0])-1
                posit = ans["end_pos"]
                ans["res"] = tn[0]
                item_ner.append(ans)
                index +=1
        test_ner.append(item_ner)
    
    return test_ner

In [ ]:
from glob import glob
test_files = glob("../data/test_data/*.txt")

In [ ]:
df_ret = {'ID':[],'Category':[],'Pos_b':[],'Pos_e':[],'Privacy':[]}
for file in test_files:
    with open(file, "r", encoding="utf-8") as f:
        line = f.read()
        line = [line]
        ret = test_predict(line, NER)
    idx = os.path.basename(file).split('.')[0]
    for line in ret[0]:
        df_ret['ID'].append(idx)
        df_ret['Category'].append(line['label_type'])
        df_ret['Pos_b'].append(line['start_pos'])
        df_ret['Pos_e'].append(line['end_pos'])
        df_ret['Privacy'].append(line['res'])

In [ ]:
version = '20201101'
df_ret_ = pd.DataFrame(df_ret)
df_ret_ = df_ret_.sort_values('ID')
df_ret_.to_csv('../submit/predict{}.csv'.format(version),index=None)

In [ ]:
file_name = '../submit/predict{}.csv'.format(version)
utils.checkout(file_name)

In [ ]:
import re
version = '1116_0.74'
file_name = '../submit/predict{}.csv'.format(version)
pred = pd.read_csv(file_name)
#pred=df_ret_
reg = re.compile(r'(《.*》)')
total_book = pred[pred.Category == 'book'][(pred[pred.Category == 'book'].Privacy.apply(lambda x:re.findall(reg,x)).apply(lambda x:len(x))!=1)]
temp = pred[pred.Category == 'book'][(pred[pred.Category == 'book'].Privacy.apply(lambda x:re.findall(reg,x)).apply(lambda x:len(x))!=1)]
bad_data_book = temp[temp.Privacy.apply(lambda x :('《' in x or '》' in x))]
print(list(bad_data_book.index))

In [10]:
def repair_book(bad_data_book,max_len = 20):
    ret = {'ID':[],'Category':[],'Pos_b':[],'Pos_e':[],'Privacy':[]}
    for idx,cat,pos_b,pos_e,privacy in bad_data_book.values:
        if '《' in privacy:
            with open('/Users/mac/Desktop/MLLearn/比赛/CCF隐私/data/test/{}.txt'.format(idx),'r') as f:
                content = f.read()
            for i in range(1, min(len(content)-pos_e-1,max_len)):
                index  = pos_e + i
                if content[index] in (',.。!！《'):
                    break
                if content[index] == '》':
                    pos_e = index
                    break
            ret['ID'].append(idx)
            ret['Category'].append(cat)
            ret['Pos_b'].append(pos_b)
            ret['Pos_e'].append(pos_e)
            ret['Privacy'].append(content[pos_b:pos_e+1])
        if '》' in privacy:
            with open('/Users/mac/Desktop/MLLearn/比赛/CCF隐私/data/test/{}.txt'.format(idx),'r') as f:
                content = f.read()
            for i in range(1,min(pos_b,max_len)):
                index  = pos_b -1 - i
                if content[index] in (',.。!！》'):
                    break
                if content[index] == '《':
                    pos_b = index
                    break
            ret['ID'].append(idx)
            ret['Category'].append(cat)
            ret['Pos_b'].append(pos_b)
            ret['Pos_e'].append(pos_e)
            ret['Privacy'].append(content[pos_b:pos_e+1])
    return ret
#l=repair_book(bad_data_book)

In [ ]:
l

In [ ]:
version = '20201122'
pred = pred.drop(list(bad_data_book.index))
l_ = pd.DataFrame(l)
pred = pred.append(l_,ignore_index=True)
pred = pred.sort_values('ID')
pred.to_csv('../submit/predict{}.csv'.format(version),index=None)

In [2]:
import re
file_name = '/Users/mac/Desktop/MLLearn/比赛/CCF隐私/course/13th_baseline介绍/DBC_code/submit/predict20201122.csv'
pred = pd.read_csv(file_name)
reg = re.compile(r'(《.*》)')
total_game = pred[pred.Category == 'game'][(pred[pred.Category == 'game'].Privacy.apply(lambda x:re.findall(reg,x)).apply(lambda x:len(x))!=1)]
temp_game = pred[pred.Category == 'game'][(pred[pred.Category == 'game'].Privacy.apply(lambda x:re.findall(reg,x)).apply(lambda x:len(x))!=1)]
bad_data_game = temp_game[temp_game.Privacy.apply(lambda x :('《' in x or '》' in x))]
total_movie = pred[pred.Category == 'movie'][(pred[pred.Category == 'movie'].Privacy.apply(lambda x:re.findall(reg,x)).apply(lambda x:len(x))!=1)]
temp_movie = pred[pred.Category == 'movie'][(pred[pred.Category == 'movie'].Privacy.apply(lambda x:re.findall(reg,x)).apply(lambda x:len(x))!=1)]
bad_data_movie = temp_movie[temp_movie.Privacy.apply(lambda x :('《' in x or '》' in x))]

In [3]:
bad_data_game

,ID,Category,Pos_b,Pos_e,Privacy
106,19,game,48,59,Kinect体感运动会》
341,52,game,0,7,爱丽丝漫游仙境》
1542,247,game,0,4,极品飞车》
1709,277,game,139,144,DOTA2》
1710,277,game,138,138,《
1989,317,game,163,172,冲突世界：苏联进攻》
2151,336,game,44,52,反恐精英零点行动》
2558,384,game,31,36,《最后的神迹
2834,425,game,32,40,生存之旅：生存包》
3611,559,game,72,77,刺客信条2》


In [4]:
bad_data_movie

,ID,Category,Pos_b,Pos_e,Privacy
372,57,movie,135,142,斯巴达三百勇士》
423,64,movie,61,66,《半盏孤灯〉
424,64,movie,67,71,《路之遥〉
562,85,movie,23,26,指环王》
1174,180,movie,191,201,《atonement）
...,...,...,...,...,...
24790,3806,movie,24,29,纳尼亚传奇》
25152,3860,movie,65,66,《说
25199,3869,movie,39,43,海角七号》
25242,3875,movie,89,90,《一


In [11]:
game=repair_book(bad_data_game)
print(game)

{'ID': [19, 52, 247, 277, 277, 317, 336, 384, 425, 559, 566, 676, 697, 733, 773, 840, 859, 922, 1094, 1116, 1145, 1192, 1231, 1231, 1359, 1473, 1715, 2008, 2245, 2285, 2296, 2422, 2422, 2424, 2526, 2651, 2802, 2805, 2834, 2834, 3000, 3041, 3120, 3294, 3407, 3462, 3534, 3541, 3563, 3563, 3636, 3813, 3910], 'Category': ['game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game', 'game'], 'Pos_b': [48, 0, 0, 139, 138, 163, 44, 31, 32, 72, 37, 96, 40, 149, 169, 46, 90, 36, 263, 0, 106, 34, 22, 22, 0, 25, 103, 60, 99, 236, 129, 37, 0, 84, 79, 202, 96, 460, 29, 28, 344, 137, 68, 228, 170, 0, 10, 81, 27, 0, 105, 47, 243], 'Pos_e': [59, 7, 4, 1

In [12]:
movie=repair_book(bad_data_movie)
print(movie)

{'ID': [57, 64, 64, 85, 180, 286, 323, 382, 410, 440, 503, 686, 792, 811, 818, 822, 863, 1017, 1039, 1110, 1121, 1197, 1198, 1246, 1498, 1611, 1616, 1704, 1736, 1775, 1799, 2012, 2055, 2126, 2128, 2157, 2202, 2217, 2257, 2300, 2300, 2302, 2306, 2416, 2424, 2491, 2568, 2809, 2809, 2930, 2931, 2969, 3113, 3134, 3139, 3147, 3192, 3210, 3378, 3404, 3413, 3466, 3497, 3497, 3543, 3571, 3580, 3634, 3648, 3652, 3658, 3689, 3741, 3806, 3860, 3869, 3875, 3935], 'Category': ['movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 'movie', 

In [13]:
version = '20201122_2'
pred = pred.drop(list(bad_data_game.index))
pred = pred.drop(list(bad_data_movie.index))
game_ = pd.DataFrame(game)
movie_ = pd.DataFrame(movie)
pred = pred.append(game_,ignore_index=True)
pred = pred.append(movie_,ignore_index=True)
pred = pred.sort_values('ID')
pred.to_csv('../submit/predict{}.csv'.format(version),index=None)